In [1]:
%%file myConv.py

import torch
import numpy as np
def myConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode != 'zeros'):
      raise Exception('only "zeros" padding_mode in ConvTranspose2d')

    #генерация ядра
    if type(kernel_size) == tuple:
      filter = torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
    if type(kernel_size) == int:
      filter = torch.rand(in_channels, out_channels, kernel_size, kernel_size)

    spisok = []

    for l in range(out_channels):

      feature_map = torch.zeros((matrix.shape[1]-1)*stride + dilation * (kernel_size-1)+1, (matrix.shape[2]-1)*stride  + dilation * (kernel_size-1)+1 ) #генерация пустой feature-map
      for c in range (in_channels):

        for i in range (0, matrix.shape[1]):  #проход по всем пикселям изображения
          for j in range (0, matrix.shape[2]):

            val = matrix[c][i][j]
            proizv = val*filter[c][l]

            zero_tensor = torch.zeros((filter.shape[2]-1)*dilation+1, (filter.shape[3]-1)*dilation+1)

            for a in range (0, zero_tensor.shape[0], dilation):
              for b in range (0, zero_tensor.shape[1], dilation):
                zero_tensor[a][b] = proizv[a//dilation][b//dilation]

            res = np.add((zero_tensor), feature_map[i*stride:i*stride+(filter.shape[2]-1)*dilation+1, j*stride:j*stride+(filter.shape[3]-1)*dilation+1])
            feature_map[i*stride:i*stride+(filter.shape[2]-1)*dilation+1, j*stride:j*stride+(filter.shape[3]-1)*dilation+1] = res


      spisok.append(np.add(feature_map, np.full((feature_map.shape), bias_val[l])))


    for t in range(len(spisok)):
      if output_padding > 0:
        pad_func = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
        spisok[t] = pad_func(spisok[t])

      spisok[t] = spisok[t][0+padding:spisok[t].shape[0]-padding, 0+padding:spisok[t].shape[1]-padding]


    return spisok, filter, torch.tensor(bias_val)

  return Svertca

Overwriting myConv.py


In [3]:
import torch
import numpy as np
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConvTranspose2d

tensor1 = torch.rand(8, 5, 6)

myFunction = myConvTranspose2d(in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction(tensor1)
torchFunction = torch.nn.ConvTranspose2d(in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val

print(result);
print('===================================')
print(str(np.array(torchFunction(tensor1).data)))

[tensor([[ 2.8598,  5.5790,  6.8660,  7.9250,  6.6737,  7.2016,  4.6999,  2.9852],
        [ 4.0509,  8.5427, 12.2801, 13.5456, 12.3468, 11.9590,  9.0198,  5.2242],
        [ 6.4449, 12.8052, 17.6082, 16.8300, 18.6578, 18.8966, 12.2961,  5.8559],
        [ 6.9400, 12.3295, 17.8442, 17.3421, 17.2084, 16.0428, 11.5736,  5.7699],
        [ 5.9667,  9.5592, 15.4848, 15.2841, 16.3996, 15.1445, 11.3261,  5.8880],
        [ 4.3850,  7.5819, 10.6721,  8.7875, 10.8828, 10.6861,  7.5851,  3.9204],
        [ 2.4671,  4.0811,  4.4687,  4.7187,  6.0020,  6.0533,  3.8547,  1.8625]]), tensor([[ 2.8483,  5.6793,  6.8157,  7.3087,  6.9912,  7.2029,  3.5701,  2.2553],
        [ 5.0666,  9.9444, 14.6408, 15.2385, 14.0816, 13.6100,  9.2088,  5.2235],
        [ 7.8791, 11.9476, 19.7869, 20.0420, 20.2104, 20.5688, 13.1608,  6.0312],
        [ 7.4884, 13.0260, 20.3440, 19.9025, 19.3294, 20.9452, 11.8518,  5.8166],
        [ 5.9635, 12.1000, 16.9360, 17.7547, 18.3462, 17.2381, 11.4345,  6.5401],
        [ 4.1

/content/myConv.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return spisok, filter, torch.tensor(bias_val)


In [4]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConvTranspose2d

tensor2 = torch.rand(3, 28, 28)

myFunction2 = myConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0, output_padding=0, dilation=3, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction2(tensor2)
torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0, output_padding=0, dilation=3, bias=True, padding_mode='zeros')
torchFunction2.weight.data = kernel
torchFunction2.bias.data = bias_val

print(result);
print('===================================')
print(str(np.array(torchFunction2(tensor2).data)))

[tensor([[0.2605, 0.1270, 0.1270,  ..., 0.1270, 0.1270, 0.6592],
        [0.1270, 0.1270, 0.1270,  ..., 0.1270, 0.1270, 0.1270],
        [0.1270, 0.1270, 0.1270,  ..., 0.1270, 0.1270, 0.1270],
        ...,
        [0.1270, 0.1270, 0.1270,  ..., 0.1270, 0.1270, 0.1270],
        [0.1270, 0.1270, 0.1270,  ..., 0.1270, 0.1270, 0.1270],
        [0.9670, 0.1270, 0.1270,  ..., 0.1270, 0.1270, 1.3622]]), tensor([[1.5108, 0.9070, 0.9070,  ..., 0.9070, 0.9070, 1.4054],
        [0.9070, 0.9070, 0.9070,  ..., 0.9070, 0.9070, 0.9070],
        [0.9070, 0.9070, 0.9070,  ..., 0.9070, 0.9070, 0.9070],
        ...,
        [0.9070, 0.9070, 0.9070,  ..., 0.9070, 0.9070, 0.9070],
        [0.9070, 0.9070, 0.9070,  ..., 0.9070, 0.9070, 0.9070],
        [1.5693, 0.9070, 0.9070,  ..., 0.9070, 0.9070, 2.0006]])]
[[[0.26048583 0.12701535 0.12701535 ... 0.12701535 0.12701535 0.65921915]
  [0.12701535 0.12701535 0.12701535 ... 0.12701535 0.12701535 0.12701535]
  [0.12701535 0.12701535 0.12701535 ... 0.12701535 0.

In [5]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConvTranspose2d

tensor3 = torch.rand(5, 6, 6)

myFunction3 = myConvTranspose2d(in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5, output_padding=2, dilation=1, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction3(tensor3)
torchFunction3 = torch.nn.ConvTranspose2d(in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5, output_padding=2, dilation=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = kernel
torchFunction3.bias.data = bias_val

print(result);
print('===================================')
print(str(np.array(torchFunction3(tensor3).data)))

[tensor([[1.7753, 1.6710, 1.5603, 1.4493, 1.7275, 1.9334, 1.7317, 1.0574, 1.7598,
         1.2063],
        [1.4878, 1.3918, 1.4311, 1.6097, 1.3737, 1.5230, 2.0465, 1.6238, 1.8384,
         1.7109],
        [1.4636, 1.5081, 1.5574, 1.3780, 1.7805, 1.6260, 1.8874, 1.9596, 1.7642,
         1.5263],
        [1.0774, 1.3967, 1.4802, 1.5431, 1.7497, 2.1047, 1.6753, 1.7384, 2.1493,
         1.9954],
        [2.1867, 1.2413, 1.6368, 1.7405, 0.8691, 0.9806, 1.5332, 1.0256, 1.1941,
         1.1755],
        [2.0014, 1.7641, 1.3868, 1.8393, 1.2281, 1.1159, 1.4325, 1.3257, 1.0744,
         1.1338],
        [2.1417, 1.4303, 2.3494, 1.2465, 1.3861, 1.6265, 1.1575, 1.1306, 1.5047,
         1.2812],
        [1.0475, 1.4424, 1.4889, 2.1523, 1.3224, 1.3409, 1.4782, 0.9236, 0.9553,
         1.4279],
        [1.0029, 1.7351, 1.7207, 1.9101, 1.2330, 1.5781, 1.2240, 1.2146, 1.0348,
         1.3188],
        [0.8476, 1.7472, 1.8946, 1.7006, 1.2214, 1.2171, 1.1202, 1.1820, 1.3157,
         1.2530]])]
[[[1.77

In [6]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConvTranspose2d

tensor4 = torch.rand(1, 1, 1)

myFunction4 = myConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction4(tensor4)
torchFunction4 = torch.nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros')
torchFunction4.weight.data = kernel
torchFunction4.bias.data = bias_val

print(result);
print('===================================')
print(str(np.array(torchFunction4(tensor4).data)))

[tensor([[0.1762]])]
[[[0.17620993]]]
